In [1]:
# 01 - Excitatory and inhibitory activity on Purkinje cell model
# Protocols to reproduce all the images based on the excitation/inhibition burst/pause behaviors
from Purkinje_morpho_1 import Purkinje_Morpho_1
from neuron import h
import multiprocessing
from plotly.subplots import make_subplots
from Purkinje_morpho_1_number import number_ind_1
import plotly.graph_objs as go
from scipy.signal import find_peaks
import numpy as np

#fixed time step only
Fixed_step = h.CVode()
Fixed_step.active(0) #the model does not work with the variable time step!

#Instantiation of the cell template
cell = Purkinje_Morpho_1()

cell_mut = Purkinje_Morpho_1()
cell_mut.soma[0].insert('na18a').gbar_na18a = 0.4
cell_mut.soma[0].gbar_Nav1_6 = 0
cell_mut.dend[0].insert('na18a').gbar_na18a = cell.dend[0].gbar_Nav1_6
cell_mut.dend[0].gbar_Nav1_6 = 0

#this code discover the number of cores available in a CPU and activate the multisplit to use them all.
cores = multiprocessing.cpu_count()
h.load_file("parcom.hoc")
p = h.ParallelComputeTool()
p.change_nthread(cores,1)
p.multisplit(1)
print ('cores', cores)

stimdata = dict()
stimdata['timeglobal'] =  10000

synapsesdata = dict()
#number of parallel fibers synapses, 1 for each section. MAX 1111
synapsesdata['npf'] = 100

#number of ascending axon synapses, 1 for each section. MAX 383
synapsesdata['naa'] = 0

#number of stellate cell synapses, 1 for each section. MAX 1332
synapsesdata['nstl'] = 0

#

#Excitation
#parallel fiber
synapsesdata['syninterval'] = 10   #distanziate di 10 = 100 hz
synapsesdata['synnumber'] = 2
synapsesdata['synstart'] = 1000
synapsesdata['synnoise'] = 0

#ascending axon
synapsesdata['synaainterval'] = 2
synapsesdata['synaanumber'] = 10
synapsesdata['synaastart'] = 600
synapsesdata['synaanoise'] = 0

#Inhibition
#Stellate on parallel fiber
synapsesdata['synstlinterval'] = 7
synapsesdata['synstlnumber'] = 10
synapsesdata['synstlstart'] = 500
synapsesdata['synstlnoise'] = 0

# delay factor
synapsesdata['synpfdelay'] = 0
synapsesdata['synaadelay'] = 0
synapsesdata['synstldelay'] = 4


3860 lines read
indiv number 53
1 1

3860 lines read
indiv number 53
1 1


NEURON: unable to open font "*helvetica-medium-r-normal*--14*", using "fixed"


cores 8


In [2]:
cell.createsyn((int(synapsesdata['npf'])), (int(synapsesdata['naa'])), (int(synapsesdata['nstl'])))
cell_mut.createsyn((int(synapsesdata['npf'])), (int(synapsesdata['naa'])), (int(synapsesdata['nstl'])))

#PF bursts
spk_stimpf = []
totalstim_pf = int(stimdata['timeglobal']/  synapsesdata['synstart'])

for j in range(int(totalstim_pf)):
    spk_stim_pf = h.NetStim()
    spk_stim_pf.interval=synapsesdata['syninterval']
    spk_stim_pf.number=synapsesdata['synnumber']
    spk_stim_pf.noise=synapsesdata['synnoise']
    spk_stim_pf.start=(synapsesdata['synstart'] * (totalstim_pf - j)) + synapsesdata['synpfdelay']
    
    spk_stimpf.append(spk_stim_pf)
    spk_nc_pfsyn = []
    j = j-1

for m in range(int(totalstim_pf)):	
    spk_nc_pfsyn.append([h.NetCon(spk_stimpf[m],PF.input,0,0.1,1) for PF in cell.PF_syn])

for m in range(int(totalstim_pf)):	
    spk_nc_pfsyn.append([h.NetCon(spk_stimpf[m],PF.input,0,0.1,1) for PF in cell_mut.PF_syn])

#Ascending Axon.
spk_stimaa = []
totalstim_aa = int(stimdata['timeglobal']/  synapsesdata['synaastart'])

for j in range(int(totalstim_aa)):
    spk_stim_aa = h.NetStim()
    spk_stim_aa.interval=synapsesdata['synaainterval']
    spk_stim_aa.number=synapsesdata['synaanumber']
    spk_stim_aa.noise=synapsesdata['synaanoise']
    spk_stim_aa.start=(synapsesdata['synaastart'] * (totalstim_aa - j)) + synapsesdata['synaadelay']
    
    spk_stimaa.append(spk_stim_aa)
    spk_nc_aasyn = []
    j = j-1

for m in range(int(totalstim_aa)):	
    spk_nc_aasyn.append([h.NetCon(spk_stimaa[m],AA.input,0,0.1,1) for AA in cell.AA_syn])
    
    
#Stellate cell
spk_stim_stl = []
totalstim_sc = int(stimdata['timeglobal']/  synapsesdata['synstlstart'])

for j in range(int(totalstim_sc)):
    spk_stim_sc = h.NetStim()
    spk_stim_sc.interval=synapsesdata['synstlinterval']
    spk_stim_sc.number=synapsesdata['synstlnumber']
    spk_stim_sc.noise=synapsesdata['synstlnoise']
    spk_stim_sc.start=(synapsesdata['synstlstart'] * (totalstim_sc - j)) + synapsesdata['synstldelay']
    
    spk_stim_stl.append(spk_stim_sc)
    spk_nc_stlsyn = []
    j = j-1


for m in range(int(totalstim_sc)):	
    spk_nc_stlsyn.append([h.NetCon(spk_stim_stl[m],stl.input,0,0.1,1) for stl in cell.stl_syn])
      

#Basic properties of the simulation. dt, temperature, sim duration and initial voltage
h.dt = 0.025
h.celsius = 32
h.tstop = 10000#stimdata['timeglobal']
h.v_init = -65

#initialization and run.    
def initialize():
    h.finitialize()
    h.run()
    
initialize()

In [ ]:
# da 20 a 80 ms input duration a 100 hz with 100 pf
# vs burst length, and burst/pause ratio

# da 5 a 20 ms input 500hz
# da 10 a 50 ms input 200hz

In [8]:
vm_h = list(cell.vm)
vm_h_a3 = list(cell.a3)

vm_m = list(cell_mut.vm)
vm_m_a3 = list(cell_mut.a3)

h = np.vstack((vm_h[len(vm_h)//2:],vm_h_a3[len(vm_h_a3)//2:]))
m = np.vstack((vm_m[len(vm_h)//2:],vm_m_a3[len(vm_h)//2:]))

np.savetxt("h_20ms_100hz", h)
np.savetxt("m_20ms_100hz", m)